<a href="https://colab.research.google.com/github/istiakm30/File-Downloader/blob/main/Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Download the file

# Import necessary libraries
import requests
from tqdm.notebook import tqdm
from google.colab import files
import re

# Create a GUI to input the URL
download_link = '' #@param {type:"string"}

# Get the file size
response = requests.get(download_link, stream=True)
total_size = int(response.headers.get('content-length', 0))

# Extract the filename from the Content-Disposition header
d = response.headers['content-disposition']
filename = re.findall("filename=(.+)", d)[0]

# Initialize a progress bar
progress_bar = tqdm(total=total_size, unit='iB', unit_scale=True)

# Download the file with progress updates
with open(filename, 'wb') as file:
    for data in response.iter_content(chunk_size=1024):
        progress_bar.update(len(data))
        file.write(data)

progress_bar.close()

if total_size != 0 and progress_bar.n != total_size:
    print("ERROR: something went wrong with the download.")
else:
    print(f"{filename} downloaded successfully.")

# Create a download link in Colab
files.download(filename)


In [ ]:
#@title Move the file to google drive's "Downloaded" folder
from google.colab import drive
import shutil
import os

# Mount Google Drive
drive.mount('/content/gdrive')

# Path to the folder where you want to save the file in your Google Drive
destination_folder = "/content/gdrive/MyDrive/Downloaded/"

# Ensure the folder exists
os.makedirs(destination_folder, exist_ok=True)

# Move the downloaded file to the Google Drive folder
shutil.move(filename, destination_folder + filename)

print(f"{filename} copied successfully to {destination_folder}")
print("File has been copied to the 'Downloaded' folder in your Google Drive.")
print("Please navigate to this folder in your Google Drive and create a shareable link if you wish to share access.")

To download files via torrents, you can either upload a .torrent file by clicking the "Choose Files" button (you need to run the code-block first) or paste a magnet link into the text box.

In [ ]:
#@title Torrent File Downloader with `.torrent` File

# Install libtorrent and tqdm
try:
  import libtorrent as lt
except:
  !apt install -y python3-libtorrent
try:
  from tqdm.notebook import tqdm
except:
  !pip install tqdm

import os
import libtorrent as lt
import time
from google.colab import files
from tqdm.notebook import tqdm

# Upload the torrent file
uploaded_torrent_file = files.upload()
torrent_file_path = list(uploaded_torrent_file.keys())[0]

# Create a hidden folder
hidden_folder_path = './.hidden_folder'
os.makedirs(hidden_folder_path, exist_ok=True)

# Move the uploaded file into the hidden folder
new_torrent_file_path = os.path.join(hidden_folder_path, torrent_file_path)
os.rename(torrent_file_path, new_torrent_file_path)

# Now, the torrent file is in the hidden folder
torrent_file_path = new_torrent_file_path

# Create a session
ses = lt.session()
info = lt.torrent_info(torrent_file_path)
h = ses.add_torrent({'ti': info, 'save_path': '.'})

# Get torrent info
total_size = info.total_size()

# Initialize progress bar
progress_bar = tqdm(total=total_size, unit='B', unit_scale=True)

# Downloading progress
print('Downloading: '+ '\033[1;32m' + '' + h.torrent_file().name() + '\033[0m')

s = h.status()
while not s.is_seeding:
    downloaded_size = s.total_done
    progress_bar.update(downloaded_size - progress_bar.n)
    peers = s.num_peers
    print('\033[1;36m' + f'\rNumber of peers: {peers}' + '\033[0m', end='')
    time.sleep(1)
    s = h.status()

print()
progress_bar.close()
print('download complete')

In [ ]:
#@title Torrent File Downloader with `Magnet Link`

# Install libtorrent and tqdm
try:
  import libtorrent as lt
except:
  !apt install -y python3-libtorrent
try:
  from tqdm.notebook import tqdm
except:
  !pip install tqdm

import libtorrent as lt
import time
from tqdm.notebook import tqdm

# Input the magnet link
magnet_link = "" #@param {type:"string"}

# Create a session
ses = lt.session()

# Add magnet link as a torrent
info_hash = lt.parse_magnet_uri(magnet_link)
info_hash.save_path = './'  # Set the path where to save the download
h = ses.add_torrent(info_hash)

# Wait for metadata to download
print('Downloading Metadata...')
while not h.torrent_file():
    time.sleep(1)
print('Metadata Downloaded')

# Get torrent info
info = h.torrent_file()
total_size = info.total_size()

# Initialize progress bar
progress_bar = tqdm(total=total_size, unit='B', unit_scale=True)

# Downloading progress
print('Downloading: '+ '\033[1;32m' + '' + info.name() + '\033[0m')

s = h.status()
while not s.is_seeding:
    downloaded_size = s.total_done
    progress_bar.update(downloaded_size - progress_bar.n)
    peers = s.num_peers
    print(f'\rNumber of peers: {peers}', end='')  # Use carriage return to overwrite the current line
    time.sleep(1)
    s = h.status()

print()  # Print a newline character after the loop to ensure proper formatting
progress_bar.close()
print('download complete')
